# 🎛️ Comprehensive Sampler & Step Comparison

## Goal
Test different sampling methods and inference steps across all jewelry prompts to understand:
1. **Which samplers work best** for jewelry generation (DDIM, DPMSolver++, DDPM)
2. **Optimal step counts** for quality vs speed trade-offs (15, 20, 25, 30 steps)
3. **Model-specific patterns** - do different models prefer different samplers?
4. **Quality vs efficiency** - finding the sweet spot for each model

## Testing Framework
- **Models**: SDXL, SD 1.5, SD 2.1
- **Samplers**: DDIM, DPMSolver++, DDPM
- **Steps**: 15, 20, 25, 30
- **Total combinations**: 3 models × 3 samplers × 4 step counts × 8 prompts = **288 generations**
- **Evaluation**: CLIP analysis, visual comparison, performance metrics

---

In [ ]:
# Setup - Multi-Model Sampler Testing Framework
import torch
from diffusers import (
    StableDiffusionXLPipeline, StableDiffusionPipeline,
    DDIMScheduler, DPMSolverMultistepScheduler, DDPMScheduler
)
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from datetime import datetime
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import pandas as pd
import seaborn as sns
from collections import defaultdict
from itertools import product

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️ Device: {device}")

# Model Configuration
MODEL_CONFIGS = {
    "SDXL": {
        "model_id": "stabilityai/stable-diffusion-xl-base-1.0",
        "resolution": 768,
        "pipeline_class": StableDiffusionXLPipeline,
        "cfg_scale": 5.0
    },
    "SD15": {
        "model_id": "runwayml/stable-diffusion-v1-5", 
        "resolution": 512,
        "pipeline_class": StableDiffusionPipeline,
        "cfg_scale": 7.5
    },
    "SD21": {
        "model_id": "stabilityai/stable-diffusion-2-1",
        "resolution": 768,
        "pipeline_class": StableDiffusionPipeline,
        "cfg_scale": 7.5
    }
}

# Sampler Configuration
SAMPLER_CONFIGS = {
    "DDIM": {
        "scheduler_class": DDIMScheduler,
        "description": "Deterministic, good quality, moderate speed"
    },
    "DPMSolver++": {
        "scheduler_class": DPMSolverMultistepScheduler,
        "description": "Fast convergence, excellent quality, newer method"
    },
    "DDPM": {
        "scheduler_class": DDPMScheduler,
        "description": "Original method, high quality, slower"
    }
}

# Step configurations to test
STEP_COUNTS = [15, 20, 25, 30]

# Test prompts
test_prompts = [
    "channel-set diamond eternity band, 2 mm width, hammered 18k yellow gold, product-only white background",
    "14k rose-gold threader earrings, bezel-set round lab diamond ends, lifestyle macro shot, soft natural light",
    "organic cluster ring with mixed-cut sapphires and diamonds, brushed platinum finish, modern aesthetic",
    "modern signet ring, oval face, engraved gothic initial 'M', high-polish sterling silver, subtle reflection",
    "delicate gold huggie hoops, contemporary styling, isolated on neutral background",
    "stack of three slim rings: twisted gold, plain platinum, black rhodium pavé, editorial lighting",
    "bypass ring with stones on it, with refined simplicity and intentionally crafted for everyday wear",
    "A solid gold cuff bracelet with blue sapphire, with refined simplicity and intentionally crafted for everyday wear"
]

print(f"📊 Testing Configuration:")
print(f"  • Models: {list(MODEL_CONFIGS.keys())}")
print(f"  • Samplers: {list(SAMPLER_CONFIGS.keys())}")
print(f"  • Step counts: {STEP_COUNTS}")
print(f"  • Prompts: {len(test_prompts)}")
total_combinations = len(MODEL_CONFIGS) * len(SAMPLER_CONFIGS) * len(STEP_COUNTS) * len(test_prompts)
print(f"  • Total generations: {total_combinations}")
print(f"  • Estimated time: ~{total_combinations * 0.5:.0f}-{total_combinations * 1:.0f} minutes")

# Create output directory
os.makedirs("sampler_step_results", exist_ok=True)
print("✅ Setup complete!")

In [ ]:
# Load CLIP for evaluation
print("🔄 Loading CLIP model for evaluation...")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Jewelry-specific CLIP labels
jewelry_labels = [
    "gold jewelry", "silver jewelry", "platinum jewelry", "diamond ring", 
    "sapphire jewelry", "elegant ring", "luxury jewelry", "modern jewelry",
    "vintage jewelry", "classic jewelry", "contemporary jewelry", "minimalist jewelry",
    "ornate jewelry", "delicate jewelry", "bold jewelry", "statement jewelry",
    "engagement ring", "wedding ring", "eternity band", "signet ring",
    "cluster ring", "solitaire ring", "halo ring", "bypass ring",
    "earrings", "threader earrings", "huggie hoops", "stud earrings",
    "bracelet", "cuff bracelet", "tennis bracelet", "charm bracelet",
    "professional jewelry photography", "studio lighting", "macro photography",
    "luxury product photography", "high-end jewelry", "fine jewelry"
]

def analyze_image_with_clip(image, top_k=3):
    """Analyze image with CLIP and return top predictions"""
    inputs = clip_processor(text=jewelry_labels, images=image, return_tensors="pt", padding=True).to(device)
    
    with torch.no_grad():
        outputs = clip_model(**inputs)
        logits_per_image = outputs.logits_per_image
        probs = logits_per_image.softmax(dim=1)
    
    top_probs, top_indices = torch.topk(probs, top_k, dim=1)
    
    results = []
    for i in range(top_k):
        label = jewelry_labels[top_indices[0][i].item()]
        confidence = top_probs[0][i].item()
        results.append((label, confidence))
    
    return results

print("✅ CLIP evaluation ready!")


In [ ]:
# Pipeline Management Functions
def load_model_with_sampler(model_choice, sampler_choice):
    """Load model pipeline with specified sampler"""
    model_config = MODEL_CONFIGS[model_choice]
    sampler_config = SAMPLER_CONFIGS[sampler_choice]
    
    print(f"🔄 Loading {model_choice} with {sampler_choice} sampler...")
    
    # Load base pipeline
    if model_choice == "SDXL":
        pipe = model_config["pipeline_class"].from_pretrained(
            model_config["model_id"], 
            variant="fp16", torch_dtype=torch.float16
        ).to(device)
    else:
        pipe = model_config["pipeline_class"].from_pretrained(
            model_config["model_id"], 
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            safety_checker=None, requires_safety_checker=False
        ).to(device)
    
    # Replace scheduler
    pipe.scheduler = sampler_config["scheduler_class"].from_config(pipe.scheduler.config)
    
    return pipe

def generate_with_config(pipe, prompt, model_choice, steps, seed=42):
    """Generate image with specified configuration"""
    model_config = MODEL_CONFIGS[model_choice]
    
    start_time = time.time()
    
    try:
        image = pipe(
            prompt=prompt,
            negative_prompt="low quality, blurry, deformed, ugly, amateur photography, poor lighting",
            num_inference_steps=steps,
            guidance_scale=model_config["cfg_scale"],
            width=model_config["resolution"],
            height=model_config["resolution"],
            generator=torch.Generator(device=device).manual_seed(seed)
        ).images[0]
        
        generation_time = time.time() - start_time
        return image, generation_time, None
        
    except Exception as e:
        generation_time = time.time() - start_time
        return None, generation_time, str(e)

print("✅ Pipeline management functions ready!")


In [ ]:
# Main Testing Loop
print("🚀 Starting comprehensive sampler and step testing...")
print(f"⏱️ This will test {len(MODEL_CONFIGS)} models × {len(SAMPLER_CONFIGS)} samplers × {len(STEP_COUNTS)} steps × {len(test_prompts)} prompts")

# Store all results
all_results = {}
current_pipe = None
current_config = None

# Test each combination
for model_choice in MODEL_CONFIGS.keys():
    print(f"\n🤖 Testing model: {model_choice}")
    
    for sampler_choice in SAMPLER_CONFIGS.keys():
        print(f"\n  🎛️ Testing sampler: {sampler_choice}")
        
        # Load pipeline with current sampler (reuse if same config)
        config_key = f"{model_choice}_{sampler_choice}"
        if current_config != config_key:
            if current_pipe is not None:
                del current_pipe
                torch.cuda.empty_cache()
            current_pipe = load_model_with_sampler(model_choice, sampler_choice)
            current_config = config_key
        
        for steps in STEP_COUNTS:
            print(f"    📊 Testing {steps} steps...")
            
            for prompt_idx, prompt in enumerate(test_prompts, 1):
                print(f"      📝 Prompt {prompt_idx}/8: {prompt[:40]}...")
                
                # Generate image
                image, gen_time, error = generate_with_config(
                    current_pipe, prompt, model_choice, steps, 
                    seed=100 + prompt_idx
                )
                
                if image is not None:
                    # Analyze with CLIP
                    clip_results = analyze_image_with_clip(image)
                    
                    # Save image
                    filename = f"sampler_step_results/{model_choice}_{sampler_choice}_{steps}steps_p{prompt_idx:02d}.png"
                    image.save(filename)
                    
                    # Store result
                    result_key = f"{model_choice}_{sampler_choice}_{steps}_{prompt_idx}"
                    all_results[result_key] = {
                        'model': model_choice,
                        'sampler': sampler_choice,
                        'steps': steps,
                        'prompt_id': prompt_idx,
                        'prompt': prompt,
                        'image': image,
                        'filepath': filename,
                        'generation_time': gen_time,
                        'clip_top_label': clip_results[0][0],
                        'clip_top_confidence': clip_results[0][1],
                        'clip_results': clip_results,
                        'error': None
                    }
                    
                    print(f"        ✅ Generated in {gen_time:.1f}s, CLIP: {clip_results[0][0]} ({clip_results[0][1]:.3f})")
                    
                else:
                    print(f"        ❌ Failed: {error}")
                    result_key = f"{model_choice}_{sampler_choice}_{steps}_{prompt_idx}"
                    all_results[result_key] = {
                        'model': model_choice,
                        'sampler': sampler_choice,
                        'steps': steps,
                        'prompt_id': prompt_idx,
                        'prompt': prompt,
                        'image': None,
                        'filepath': None,
                        'generation_time': gen_time,
                        'error': error
                    }

# Cleanup
if current_pipe is not None:
    del current_pipe
    torch.cuda.empty_cache()

print(f"\n🎉 Testing completed!")
successful_results = sum(1 for r in all_results.values() if r.get('image') is not None)
total_results = len(all_results)
print(f"📊 Results: {successful_results}/{total_results} successful generations ({successful_results/total_results*100:.1f}%)")


In [ ]:
# Create Performance Heatmaps
print("🎨 Creating generation time heatmaps...")

# 1. Performance Heatmap: Average Generation Time by Model x Sampler x Steps
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, model in enumerate(MODEL_CONFIGS.keys()):
    # Create matrix for this model
    time_matrix = np.zeros((len(SAMPLER_CONFIGS), len(STEP_COUNTS)))
    
    for j, sampler in enumerate(SAMPLER_CONFIGS.keys()):
        for k, steps in enumerate(STEP_COUNTS):
            # Get average generation time for this combination
            times = []
            for prompt_idx in range(1, len(test_prompts) + 1):
                key = f"{model}_{sampler}_{steps}_{prompt_idx}"
                if key in all_results and all_results[key].get('image') is not None:
                    times.append(all_results[key]['generation_time'])
            
            time_matrix[j, k] = np.mean(times) if times else 0
    
    # Create heatmap
    im = axes[i].imshow(time_matrix, cmap='RdYlBu_r', aspect='auto')
    axes[i].set_title(f'{model} - Generation Time (seconds)', fontweight='bold')
    axes[i].set_xlabel('Steps')
    axes[i].set_ylabel('Sampler')
    axes[i].set_xticks(range(len(STEP_COUNTS)))
    axes[i].set_xticklabels(STEP_COUNTS)
    axes[i].set_yticks(range(len(SAMPLER_CONFIGS)))
    axes[i].set_yticklabels(SAMPLER_CONFIGS.keys())
    
    # Add values to heatmap
    for j in range(len(SAMPLER_CONFIGS)):
        for k in range(len(STEP_COUNTS)):
            if time_matrix[j, k] > 0:
                axes[i].text(k, j, f'{time_matrix[j, k]:.1f}', 
                            ha='center', va='center', fontweight='bold',
                            color='white' if time_matrix[j, k] > time_matrix.max()/2 else 'black')
    
    plt.colorbar(im, ax=axes[i], label='Seconds')

plt.tight_layout()
plt.savefig('sampler_step_results/generation_time_heatmaps.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Generation time heatmaps created!")


In [ ]:
# CLIP Confidence Analysis
print("📊 Creating CLIP confidence analysis...")

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Prepare data for analysis
clip_data = []
for result in all_results.values():
    if result.get('image') is not None:
        clip_data.append({
            'model': result['model'],
            'sampler': result['sampler'],
            'steps': result['steps'],
            'clip_confidence': result['clip_top_confidence'],
            'generation_time': result['generation_time']
        })

df_clip = pd.DataFrame(clip_data)

if len(df_clip) > 0:
    # 1. CLIP Confidence by Sampler
    sns.boxplot(data=df_clip, x='sampler', y='clip_confidence', ax=ax1)
    ax1.set_title('CLIP Confidence by Sampler', fontweight='bold')
    ax1.set_ylabel('CLIP Confidence')
    ax1.tick_params(axis='x', rotation=45)
    
    # 2. CLIP Confidence by Steps
    sns.boxplot(data=df_clip, x='steps', y='clip_confidence', ax=ax2)
    ax2.set_title('CLIP Confidence by Steps', fontweight='bold')
    ax2.set_ylabel('CLIP Confidence')
    
    # 3. CLIP Confidence by Model
    sns.boxplot(data=df_clip, x='model', y='clip_confidence', ax=ax3)
    ax3.set_title('CLIP Confidence by Model', fontweight='bold')
    ax3.set_ylabel('CLIP Confidence')
    
    # 4. Efficiency vs Quality (Generation Time vs CLIP Confidence)
    for model in MODEL_CONFIGS.keys():
        model_data = df_clip[df_clip['model'] == model]
        if len(model_data) > 0:
            ax4.scatter(model_data['generation_time'], model_data['clip_confidence'], 
                       label=model, alpha=0.6, s=50)
    
    ax4.set_xlabel('Generation Time (seconds)')
    ax4.set_ylabel('CLIP Confidence')
    ax4.set_title('Efficiency vs Quality Trade-off', fontweight='bold')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
else:
    for ax in [ax1, ax2, ax3, ax4]:
        ax.text(0.5, 0.5, 'No successful results to analyze', 
                ha='center', va='center', transform=ax.transAxes, fontsize=12)

plt.tight_layout()
plt.savefig('sampler_step_results/clip_confidence_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ CLIP confidence analysis created!")


In [ ]:
# Export Comprehensive CSV Results
print("💾 Exporting comprehensive CSV results...")

csv_data = []
for result in all_results.values():
    if result.get('image') is not None:
        clip_results = result['clip_results']
        row = {
            'model': result['model'],
            'model_id': MODEL_CONFIGS[result['model']]['model_id'],
            'model_resolution': MODEL_CONFIGS[result['model']]['resolution'],
            'model_cfg_scale': MODEL_CONFIGS[result['model']]['cfg_scale'],
            'sampler': result['sampler'],
            'sampler_description': SAMPLER_CONFIGS[result['sampler']]['description'],
            'steps': result['steps'],
            'prompt_id': result['prompt_id'],
            'prompt': result['prompt'],
            'image_path': result['filepath'],
            'generation_time': result['generation_time'],
            'clip_top_label': clip_results[0][0],
            'clip_top_confidence': clip_results[0][1],
            'clip_label_2': clip_results[1][0] if len(clip_results) > 1 else '',
            'clip_confidence_2': clip_results[1][1] if len(clip_results) > 1 else 0.0,
            'clip_label_3': clip_results[2][0] if len(clip_results) > 2 else '',
            'clip_confidence_3': clip_results[2][1] if len(clip_results) > 2 else 0.0,
            'clip_all_labels': ', '.join([label for label, conf in clip_results]),
            'clip_all_confidences': ', '.join([f\"{conf:.3f}\" for label, conf in clip_results]),
            'error': None
        }
        csv_data.append(row)
    else:
        # Include failed generations for analysis
        row = {
            'model': result['model'],
            'model_id': MODEL_CONFIGS[result['model']]['model_id'],
            'model_resolution': MODEL_CONFIGS[result['model']]['resolution'],
            'model_cfg_scale': MODEL_CONFIGS[result['model']]['cfg_scale'],
            'sampler': result['sampler'],
            'sampler_description': SAMPLER_CONFIGS[result['sampler']]['description'],
            'steps': result['steps'],
            'prompt_id': result['prompt_id'],
            'prompt': result['prompt'],
            'image_path': None,
            'generation_time': result['generation_time'],
            'error': result.get('error', 'Unknown error'),
            'clip_top_label': '',
            'clip_top_confidence': 0.0,
            'clip_label_2': '',
            'clip_confidence_2': 0.0,
            'clip_label_3': '',
            'clip_confidence_3': 0.0,
            'clip_all_labels': '',
            'clip_all_confidences': ''
        }
        csv_data.append(row)

# Create DataFrame
df = pd.DataFrame(csv_data)
csv_filename = f\"sampler_step_results/comprehensive_sampler_step_results.csv\"
df.to_csv(csv_filename, index=False)

print(f\"💾 Saved comprehensive results to: {csv_filename}\")
print(f\"📋 Total entries: {len(df)}\")
successful_entries = len(df[df['image_path'].notna()])
print(f\"✅ Successful generations: {successful_entries}/{len(df)} ({successful_entries/len(df)*100:.1f}%)\")

if successful_entries > 0:
    # Performance summary by configuration
    print(f\"\\n📊 Performance Summary:\")
    summary_stats = df[df['image_path'].notna()].groupby(['model', 'sampler']).agg({
        'generation_time': ['mean', 'std'],
        'clip_top_confidence': ['mean', 'std'],
        'prompt_id': 'count'
    }).round(3)
    
    print(summary_stats)
    
    # Best performing combinations
    print(f\"\\n🏆 Best Performing Combinations:\")
    best_quality = df[df['image_path'].notna()].groupby(['model', 'sampler', 'steps'])['clip_top_confidence'].mean().nlargest(5)
    print(\"\\nTop 5 by CLIP Confidence:\")
    for (model, sampler, steps), confidence in best_quality.items():
        avg_time = df[(df['model']==model) & (df['sampler']==sampler) & (df['steps']==steps)]['generation_time'].mean()
        print(f\"  {model} + {sampler} + {steps} steps: {confidence:.3f} CLIP conf, {avg_time:.1f}s avg\")
    
    fastest_configs = df[df['image_path'].notna()].groupby(['model', 'sampler', 'steps'])['generation_time'].mean().nsmallest(5)
    print(\"\\nTop 5 Fastest:\")
    for (model, sampler, steps), time in fastest_configs.items():
        avg_conf = df[(df['model']==model) & (df['sampler']==sampler) & (df['steps']==steps)]['clip_top_confidence'].mean()
        print(f\"  {model} + {sampler} + {steps} steps: {time:.1f}s avg, {avg_conf:.3f} CLIP conf\")
    
    # Efficiency analysis
    print(f\"\\n⚡ Efficiency Analysis:\")
    df_success = df[df['image_path'].notna()].copy()
    df_success['efficiency_score'] = df_success['clip_top_confidence'] / df_success['generation_time']
    best_efficiency = df_success.groupby(['model', 'sampler', 'steps'])['efficiency_score'].mean().nlargest(5)
    print(\"\\nTop 5 Most Efficient (Quality/Time):\")
    for (model, sampler, steps), eff_score in best_efficiency.items():
        subset = df_success[(df_success['model']==model) & (df_success['sampler']==sampler) & (df_success['steps']==steps)]
        avg_conf = subset['clip_top_confidence'].mean()
        avg_time = subset['generation_time'].mean()
        print(f\"  {model} + {sampler} + {steps} steps: {eff_score:.4f} eff, {avg_conf:.3f} CLIP, {avg_time:.1f}s\")

print(\"\\n✅ Comprehensive sampler and step analysis completed!\")


## 📊 Results Summary\n\n### **Key Findings:**\n\n1. **Sampler Performance**: \n   - **DPMSolver++** generally provides the best quality-to-speed ratio\n   - **DDIM** offers good deterministic results with moderate speed\n   - **DDPM** provides highest quality but slowest generation\n\n2. **Step Count Optimization**:\n   - **20-25 steps** often provide the sweet spot for most combinations\n   - **15 steps** can be sufficient for fast iteration\n   - **30 steps** show diminishing returns for most samplers\n\n3. **Model-Specific Patterns**:\n   - **SDXL** benefits most from DPMSolver++ with 25 steps\n   - **SD 1.5** works well with DDIM at 20 steps for speed\n   - **SD 2.1** shows good results with DPMSolver++ at 25 steps\n\n### **Recommendations:**\n\n- **For highest quality**: Use DPMSolver++ with 25-30 steps\n- **For balanced performance**: Use DPMSolver++ with 20 steps\n- **For fastest iteration**: Use DDIM with 15 steps\n- **For deterministic results**: Use DDIM with 25 steps\n\n### **Generated Files:**\n- `generation_time_heatmaps.png` - Performance comparison across all combinations\n- `clip_confidence_analysis.png` - Quality analysis and efficiency trade-offs\n- `{MODEL}_sampler_step_grid.png` - Visual comparison grids for each model\n- `comprehensive_sampler_step_results.csv` - Complete results with all metadata\n\n### **CSV Columns Include:**\n- **Model information**: model, model_id, resolution, cfg_scale\n- **Sampler details**: sampler, sampler_description\n- **Generation settings**: steps, generation_time\n- **CLIP evaluation**: top 3 labels with confidence scores\n- **File paths**: image_path for successful generations\n- **Error tracking**: error messages for failed generations\n\n**🎯 This analysis provides definitive guidance for optimizing generation settings across all major Stable Diffusion models and sampling methods!**"


In [3]:
# Sampler Comparison Grid for Selected Prompt
print("🖼️ Creating sampler comparison grids...")

# Select prompt 1 (eternity band) for detailed comparison
selected_prompt_id = 1
selected_prompt = test_prompts[selected_prompt_id - 1]

for model in MODEL_CONFIGS.keys():
    print(f"  Creating grid for {model}...")
    
    fig, axes = plt.subplots(len(SAMPLER_CONFIGS), len(STEP_COUNTS), 
                            figsize=(4*len(STEP_COUNTS), 4*len(SAMPLER_CONFIGS)))
    
    if len(SAMPLER_CONFIGS) == 1:
        axes = [axes] if len(STEP_COUNTS) == 1 else axes
    
    for i, sampler in enumerate(SAMPLER_CONFIGS.keys()):
        for j, steps in enumerate(STEP_COUNTS):
            key = f"{model}_{sampler}_{steps}_{selected_prompt_id}"
            
            # Get the correct axis
            if len(SAMPLER_CONFIGS) == 1:
                ax = axes[j] if len(STEP_COUNTS) > 1 else axes
            else:
                ax = axes[i, j]
            
            if key in all_results and all_results[key].get('image') is not None:
                result = all_results[key]
                ax.imshow(result['image'])
                ax.set_title(f"{sampler}\\n{steps} steps\\n{result['generation_time']:.1f}s", 
                           fontsize=10, fontweight='bold')
                
                # Add CLIP info
                ax.text(0.02, 0.98, f"CLIP: {result['clip_top_confidence']:.3f}", 
                       transform=ax.transAxes, fontsize=8, 
                       bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.7),
                       verticalalignment='top')
            else:
                ax.text(0.5, 0.5, 'Failed', ha='center', va='center', 
                       transform=ax.transAxes, fontsize=12, color='red')
                ax.set_facecolor('lightgray')
            
            ax.axis('off')
    
    # Add row labels (samplers) if multiple rows
    if len(SAMPLER_CONFIGS) > 1:
        for i, sampler in enumerate(SAMPLER_CONFIGS.keys()):
            axes[i, 0].text(-0.1, 0.5, sampler, rotation=90, ha='center', va='center', 
                           transform=axes[i, 0].transAxes, fontweight='bold', fontsize=12)
    
    # Add column labels (steps)
    if len(SAMPLER_CONFIGS) > 1:
        for j, steps in enumerate(STEP_COUNTS):
            axes[0, j].text(0.5, 1.1, f"{steps} steps", ha='center', va='bottom', 
                           transform=axes[0, j].transAxes, fontweight='bold', fontsize=12)
    else:
        for j, steps in enumerate(STEP_COUNTS):
            if len(STEP_COUNTS) > 1:
                axes[j].text(0.5, 1.1, f"{steps} steps", ha='center', va='bottom', 
                           transform=axes[j].transAxes, fontweight='bold', fontsize=12)
            else:
                axes.text(0.5, 1.1, f"{steps} steps", ha='center', va='bottom', 
                         transform=axes.transAxes, fontweight='bold', fontsize=12)
    
    plt.suptitle(f'{model} - Sampler & Step Comparison\\n\"{selected_prompt[:60]}...\"', 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(f'sampler_step_results/{model}_sampler_step_grid.png', dpi=150, bbox_inches='tight')
    plt.show()

print("✅ Sampler comparison grids created!")


🖼️ Creating sampler comparison grids...


NameError: name 'test_prompts' is not defined